Tasks
=====

- [x] Get test FASTQ
- [ ] Get test FASTA
- [ ] Note actual marker call
- Write various kmer selection strategies
    - [x] Exhaustive
    - [x] Flanks
    - [x] End-to-end
    - [ ] Variable
    - [ ] Conserved
- [ ] Extract reads matching kmers


Notes
=====

- A-C seems to take the same amount of time regardless of kmer length
- KAT gets faster on longer kmers

In [24]:
import timeit
from Bio import SeqIO
from pathlib import Path
import subprocess
from typing import Set
from ahocorasick import Automaton
import pandas as pd
from collections import defaultdict

In [8]:
def yield_fasta_sequences(alleles_path: Path):
    with alleles_path.open('r') as f:
        for record in SeqIO.parse(f, 'fasta'):
            yield str(record.seq)

In [9]:
def kmer_to_fasta(kmers: Set[str]):
    
    return '\n'.join(f'>{i}\n{s}' for i, s in enumerate(kmers))

In [19]:
def init_automaton(scheme_fasta):
    """Initialize Aho-Corasick Automaton with kmers from SNV scheme fasta
    Args:
        scheme_fasta: SNV scheme fasta file path
    Returns:
         Aho-Corasick Automaton with kmers loaded
    """
    A = Automaton()
    with scheme_fasta.open('r') as f:
        for record in SeqIO.parse(f, 'fasta'):
            header = record.id
            seq = str(record.seq)
            A.add_word(seq, (header, seq, False))
            A.add_word(revcomp(seq), (header, seq, True))
    A.make_automaton()
    return A

In [22]:
def revcomp(seq):
    complements = {
        'A': 'T',
        'T': 'A',
        'G': 'C',
        'C': 'G'
    }
    
    return ''.join(complements[x] for x in reversed(seq))


In [11]:
def find_in_fastqs(A: Automaton, *fastqs):
    """Find scheme kmers in input fastq files
    Args:
        A: Aho-Corasick Automaton with scheme SNV target kmers loaded
        fastqs: Input fastq file paths
    Returns:
        Dataframe with any matches found in input fastq files
    """
    kmer_seq_counts = defaultdict(int)
    for fastq in fastqs:
        with fastq.open('r') as f:
               
            for record in SeqIO.parse(f, 'fastq'):
                sequence = str(record.seq)
                for idx, (_, kmer_seq, _) in A.iter(sequence):
                    kmer_seq_counts[kmer_seq] += 1
                
    res = []
    for kmer_seq, freq in kmer_seq_counts.items():
        kmername, sequence, _ = A.get(kmer_seq)
        res.append((kmername, kmer_seq, freq))
  
    df = pd.DataFrame(res, columns=['kmername', 'seq', 'freq'])
    return df

In [12]:
def kat_filter(kmer_fasta: Path, fwd_fastq: Path, rev_fastq: Path, kmer_size: int):
    
    cmd = (
        'kat', 'filter', 'seq',
        '-t', '1',
        '-m', str(kmer_size),
        '-o', 'testo',
        '--seq', str(fwd_fastq),
        '--seq2', str(rev_fastq),
        str(kmer_fasta)
    )
    
    subprocess.run(cmd)

In [39]:
def exhaustive(alleles_path: Path, kmer_size: int):
    kmers = set()
    for seq in yield_fasta_sequences(alleles_path):
        
        seq_length = len(seq)
        start = 0
        end = start + kmer_size
        
        while end < seq_length:
            kmers.add(seq[start : end])
        
    return kmers

In [14]:
def flanks(alleles_path: Path, kmer_size: int):
    kmers = set()
    for seq in yield_fasta_sequences(alleles_path):
        
        kmers.add(seq[0 : kmer_size])
        kmers.add(seq[-kmer_size : ])
        
    return kmers

In [15]:
def end_to_end(alleles_path: Path, kmer_size):
    
    kmers = set()
    
    for seq in yield_fasta_sequences(alleles_path):
        
        seq_length = len(seq)
        breaks = range(0, seq_length, kmer_size)
        
        kmers_ = set(seq[i : i + kmer_size] for i in breaks if i + kmer_size < seq_length)
        
        kmers = kmers.union(kmers_)
        
    return kmers
        

In [47]:
alleles = Path("test/aarA.fasta")
test_kmers = Path('test/test_kmers.fasta')
kmers = end_to_end(alleles, 125)
fwd, rev = Path('test/CI-5429/').glob('*.fastq')
test_kmers.write_text(kmer_to_fasta(kmers))

119492

In [48]:
automaton = init_automaton(test_kmers)
%time res = find_in_fastqs(automaton, fwd, rev)

CPU times: user 28.3 s, sys: 181 ms, total: 28.5 s
Wall time: 28 s


In [49]:
print(res)

    kmername                                                seq  freq
0        607  GATTATTACAATATCCTTGGTTTGAATATTTTATTTTTTAATGGAG...     8
1        661  TTATTACAATATCCTTGGTTTGAATATTTTATTTTTTAATGGAGCT...     8
2        664  TATTACAATATCCTTGGTTTGAATATTTTATTTTTTAATGGAGCTT...     9
3        299  TTACAATATCCTTGGTTTGAATATTTTATTTTTTAATGGAGCTTAT...    10
4        579  TACAATATCCTTGGTTTGAATATTTTATTTTTTAATGGAGCTTATT...    10
..       ...                                                ...   ...
123      902  TTTATTTATTTTATTGGCGGATTGTTGTGCTCACTTTTAAGTGTTT...     4
124      150  TTGCTTACAATTTTTTTAATTTTTTTAAATATTTTATGTTATTTTT...     2
125      171  GCTTACAATTTTTTTAATTTTTTTAAATATTTTATGTTATTTTTTG...     2
126      518  CAATTTTTTTAATTTTTTTAAATATTTTATGTTATTTTTTGATTTC...     2
127      162  AATTTTTTTAATTTTTTTAAATATTTTATGTTATTTTTTGATTTCT...     2

[128 rows x 3 columns]


In [51]:
sum(res.freq)

763